In [10]:
import pandas as pd
import numpy as np
import os

import math

import src.visualization.simple_visualization as sv
import src.features.clean as cl
import src.features.feature_engineering_1 as feng

In [19]:
# Récupération des données
#years = ['2016', '2017', '2018', '2019', '2020']
years = ['2016']
df = feng.get_all_season_data(years)
df.head()

,period,period_type,period_time,gameID,attacking_team_id,attacking_team_name,play_type,shooter,goalie,shot_type,x_coordinate,y_coordinate,empty_net,strength,home_team,last_event_type,last_event_x,last_event_y,time_since_last_event,distance_from_last_event
0,1,REGULAR,01:11,2016020001,10,Toronto Maple Leafs,Shot,Mitchell Marner,Craig Anderson,Wrist Shot,-77.0,5.0,NaN,NaN,Ottawa Senators,Blocked Shot,-61.0,11.0,1,17.088007
1,1,REGULAR,02:53,2016020001,9,Ottawa Senators,Shot,Chris Kelly,Frederik Andersen,Wrist Shot,86.0,13.0,NaN,NaN,Ottawa Senators,Giveaway,54.0,-5.0,5,36.715120
2,1,REGULAR,04:01,2016020001,9,Ottawa Senators,Shot,Cody Ceci,Frederik Andersen,Wrist Shot,23.0,-38.0,NaN,NaN,Ottawa Senators,Missed Shot,-72.0,0.0,18,102.318131
3,1,REGULAR,04:46,2016020001,9,Ottawa Senators,Shot,Erik Karlsson,Frederik Andersen,Slap Shot,33.0,-15.0,NaN,NaN,Ottawa Senators,Missed Shot,77.0,-2.0,19,45.880279
4,1,REGULAR,06:46,2016020001,10,Toronto Maple Leafs,Shot,Martin Marincin,Craig Anderson,Wrist Shot,-34.0,28.0,NaN,NaN,Ottawa Senators,Hit,47.0,34.0,16,81.221918


In [20]:
df = feng.add_new_features(df)
df.head()

,period,period_type,period_time,gameID,attacking_team_id,attacking_team_name,play_type,shooter,goalie,shot_type,...,home_team,last_event_type,last_event_x,last_event_y,time_since_last_event,distance_from_last_event,distance_to_net,shot_angle,is_goal,is_empty_net
0,1,REGULAR,01:11,2016020001,10,Toronto Maple Leafs,Shot,Mitchell Marner,Craig Anderson,Wrist Shot,...,Ottawa Senators,Blocked Shot,-61.0,11.0,1,17.088007,13.0,23.0,0,0
1,1,REGULAR,02:53,2016020001,9,Ottawa Senators,Shot,Chris Kelly,Frederik Andersen,Wrist Shot,...,Ottawa Senators,Giveaway,54.0,-5.0,5,36.715120,13.0,77.0,0,0
2,1,REGULAR,04:01,2016020001,9,Ottawa Senators,Shot,Cody Ceci,Frederik Andersen,Wrist Shot,...,Ottawa Senators,Missed Shot,-72.0,0.0,18,102.318131,76.0,30.0,0,0
3,1,REGULAR,04:46,2016020001,9,Ottawa Senators,Shot,Erik Karlsson,Frederik Andersen,Slap Shot,...,Ottawa Senators,Missed Shot,77.0,-2.0,19,45.880279,58.0,15.0,0,0
4,1,REGULAR,06:46,2016020001,10,Toronto Maple Leafs,Shot,Martin Marincin,Craig Anderson,Wrist Shot,...,Ottawa Senators,Hit,47.0,34.0,16,81.221918,62.0,27.0,0,0


### Question 1

In [21]:
df['game_seconds'] = df.apply(lambda x : cl.get_game_seconds(x['period'], x['period_time']), axis = 1)
df.drop(['empty_net', 'play_type'], axis=1, inplace = True)
df.head()

,period,period_type,period_time,gameID,attacking_team_id,attacking_team_name,shooter,goalie,shot_type,x_coordinate,...,last_event_type,last_event_x,last_event_y,time_since_last_event,distance_from_last_event,distance_to_net,shot_angle,is_goal,is_empty_net,game_seconds
0,1,REGULAR,01:11,2016020001,10,Toronto Maple Leafs,Mitchell Marner,Craig Anderson,Wrist Shot,-77.0,...,Blocked Shot,-61.0,11.0,1,17.088007,13.0,23.0,0,0,71
1,1,REGULAR,02:53,2016020001,9,Ottawa Senators,Chris Kelly,Frederik Andersen,Wrist Shot,86.0,...,Giveaway,54.0,-5.0,5,36.715120,13.0,77.0,0,0,173
2,1,REGULAR,04:01,2016020001,9,Ottawa Senators,Cody Ceci,Frederik Andersen,Wrist Shot,23.0,...,Missed Shot,-72.0,0.0,18,102.318131,76.0,30.0,0,0,241
3,1,REGULAR,04:46,2016020001,9,Ottawa Senators,Erik Karlsson,Frederik Andersen,Slap Shot,33.0,...,Missed Shot,77.0,-2.0,19,45.880279,58.0,15.0,0,0,286
4,1,REGULAR,06:46,2016020001,10,Toronto Maple Leafs,Martin Marincin,Craig Anderson,Wrist Shot,-34.0,...,Hit,47.0,34.0,16,81.221918,62.0,27.0,0,0,406


### Question 2
Les modifications nécessaires, sont apportés à clean.py. Il suffit de réxecuter le script.

### Question 3

In [27]:
def add_features(df: pd.DataFrame):
    # Rebond
    df['rebound'] = df['last_event_type']=='Shot'

    # Changement d'angle
    df['angle_difference'] = df['shot_angle'].diff().fillna(0)
    df['angle_change'] = np.where(df['rebound'], df['angle_difference'], 0)
    df.drop(['angle_difference'], axis = 1, inplace= True)

    # Vitesse
    df['speed'] = df['distance_from_last_event'] / df['time_since_last_event'].replace({0: np.nan})
    return df

In [28]:
df = df.add_features(df)
df.head()

,period,period_type,period_time,gameID,attacking_team_id,attacking_team_name,shooter,goalie,shot_type,x_coordinate,...,time_since_last_event,distance_from_last_event,distance_to_net,shot_angle,is_goal,is_empty_net,game_seconds,rebound,angle_change,speed
0,1,REGULAR,01:11,2016020001,10,Toronto Maple Leafs,Mitchell Marner,Craig Anderson,Wrist Shot,-77.0,...,1,17.088007,13.0,23.0,0,0,71,False,0.0,17.088007
1,1,REGULAR,02:53,2016020001,9,Ottawa Senators,Chris Kelly,Frederik Andersen,Wrist Shot,86.0,...,5,36.715120,13.0,77.0,0,0,173,False,0.0,7.343024
2,1,REGULAR,04:01,2016020001,9,Ottawa Senators,Cody Ceci,Frederik Andersen,Wrist Shot,23.0,...,18,102.318131,76.0,30.0,0,0,241,False,0.0,5.684341
3,1,REGULAR,04:46,2016020001,9,Ottawa Senators,Erik Karlsson,Frederik Andersen,Slap Shot,33.0,...,19,45.880279,58.0,15.0,0,0,286,False,0.0,2.414752
4,1,REGULAR,06:46,2016020001,10,Toronto Maple Leafs,Martin Marincin,Craig Anderson,Wrist Shot,-34.0,...,16,81.221918,62.0,27.0,0,0,406,False,0.0,5.076370


In [29]:
df[df['rebound']==True].head()

,period,period_type,period_time,gameID,attacking_team_id,attacking_team_name,shooter,goalie,shot_type,x_coordinate,...,time_since_last_event,distance_from_last_event,distance_to_net,shot_angle,is_goal,is_empty_net,game_seconds,rebound,angle_change,speed
10,1,REGULAR,10:25,2016020001,9,Ottawa Senators,Erik Karlsson,Frederik Andersen,Slap Shot,34.0,...,9,45.000000,60.0,24.0,0,0,625,True,4.0,5.000000
11,1,REGULAR,10:26,2016020001,9,Ottawa Senators,Bobby Ryan,Frederik Andersen,Backhand,82.0,...,1,55.569776,8.0,23.0,1,0,626,True,-1.0,55.569776
14,1,REGULAR,12:49,2016020001,9,Ottawa Senators,Erik Karlsson,Frederik Andersen,Slap Shot,34.0,...,5,35.693137,55.0,1.0,1,0,769,True,-21.0,7.138627
23,2,REGULAR,01:03,2016020001,9,Ottawa Senators,Jean-Gabriel Pageau,Frederik Andersen,Wrist Shot,-55.0,...,9,100.179838,40.0,32.0,0,0,963,True,0.0,11.131093
26,2,REGULAR,02:08,2016020001,9,Ottawa Senators,Mark Stone,Frederik Andersen,Backhand,-87.0,...,6,58.591808,8.0,76.0,0,0,1028,True,57.0,9.765301
